# 계층적 군집의 덴드로그램

## #01. 준비작업

### [1] 패키지 가져오기

In [ ]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.analysis import *

from sklearn.cluster import AgglomerativeClustering

from scipy.cluster.hierarchy import dendrogram, linkage

### [2] 데이터 가져오기 + 스케일링

In [ ]:
origin = my_read_excel("https://data.hossam.kr/mldata/game_usage.xlsx", info=False)
df = my_standard_scaler(origin)
my_pretty_table(df.head())

## #02. 군집 수행

### [1] scipy 패키지를 사용한 계층적 군집

#### (1) 군집 모형 구현

scipy 패키지에서는 병합군집 기능을 제공한다.

군집 결과를 2차원 배열로 리턴해 준다.

> 여기서는 하이퍼파라미터는 모두 기본값 상태로 수행함

In [ ]:
lnk = linkage(df)
lnk

#### (2) 군집 결과 시각화

In [ ]:
fig = plt.figure(figsize=(21, 7), dpi=200)
ax = fig.gca()

dendrogram(lnk, ax=ax, 
           leaf_rotation=0, leaf_font_size=12, 
            # 각 노드 n에 대해 두 하위 링크가 표시되는 순서
           # False: 아무것도 안함(기본값)
           # 'ascending': 클러스터에 원본 개체수가 가장 적은 하위 객체가 먼저 출력
           # `descending`: 클러스터에 원본 개체수가 가장 많은 하위 객체가 먼저 출력
           count_sort='ascending', 
           # 각 노드 n에 대해 두 하위 링크가 표시되는 순서
           # False: 아무것도 안함(기본값)
           # 'ascending': 직계 자손의 사이의 거리가 최소인 하위 항목이 먼저 표시됨
           # `descending`: 직계 자손의 사이의 거리가 최대인 하위 항목이 먼저 표시됨
           distance_sort='ascending',
           show_leaf_counts=True, # True: 맨 아래에 노드에 속한 개체 수 표시(k>1 경우만...)
)

ax.set_title("Dendrogram")
plt.show()
plt.close()

### [2] sklearn을 사용한 군집

#### (1) 군집 모델 구현

In [ ]:
estimator = AgglomerativeClustering(n_clusters=4,
                                    # 노드간의 거리 계산값을 도출할지 여부 (시각화를 위해서는 필수 적용)
                                    compute_distances=True)

estimator.fit(df)

result_df = origin.copy()
result_df['cluster'] = estimator.labels_

my_pretty_table(result_df.head(10))

#### (3) 군집 결과 시각화

`estimator`객체가 포함하고 있는 여러 프로퍼티들을 조합해서 scipy 패키지의 리턴값과 동일한 형태로 구성해주는 함수

> 출처: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html

In [ ]:
def my_dandrogram_source(estimator):
    counts = np.zeros(estimator.children_.shape[0])
    n_samples = len(estimator.labels_)

    for i, merge in enumerate(estimator.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [estimator.children_, estimator.distances_, counts]
    ).astype(float)
    
    # 시각화에 필요한 배열 리턴
    return linkage_matrix

#### (4) 함수를 통해 리턴받은 값을 시각화

In [ ]:
data = my_dandrogram_source(estimator)

fig = plt.figure(figsize=(21, 7), dpi=200)
ax = fig.gca()

dendrogram(data, show_leaf_counts=True, ax=ax, leaf_rotation=0, leaf_font_size=12, show_contracted=True)

ax.set_title("Dendrogram")

plt.show()
plt.close()

In [ ]:
my_convex_hull(result_df, xname='time spent', yname='game level', hue='cluster', palette='Set1')